# Problem 5.3  

**Integrated Energy Grids**

**Problem 5.3**

_Note: This is equivalent to Problem 2.2 which we solved using linopy, but here we will use PyPSA._

**Consider the following economic dispatch problem:**

**• we have three generators: solar, wind and gas**

**• solar and wind have no marginal costs, and gas has fuel costs of 60 EUR/MWh.**

**• we need to cover a electricity demand of 13.2 MWh**

**• the installed capacities are 15 MW, 20 MW and 20 MW for wind, solar, and gas, respectively**

**• assume the capacity factor for solar is 0.17 and for wind 0.33.**

**Use PYPSA to find the optimal solution as well and obtain the electricity price (Lagrange multiplier for the energy balance constraint)**



:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install numpy pypsa
```
:::

In [1]:
import numpy as np
import pypsa

Set parameter Username
Set parameter LicenseID to value 2767832
Academic license - for non-commercial use only - expires 2027-01-20


We start by creating the network object and adding the bus.

In [2]:
network = pypsa.Network()

In [3]:
network.add("Bus", "my bus") 
network.buses

,v_nom,type,x,y,carrier,unit,location,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
name,,,,,,,,,,,,,
my bus,1.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,


We add the generators

In [4]:
network.add("Generator", 
            "solar", 
            bus="my bus", 
            p_nom=15, 
            p_max_pu=0.17, #capacity factor
            marginal_cost=0) 
    
network.add("Generator", 
            "wind", 
            bus="my bus", 
            p_nom=20, 
            p_max_pu=0.33, #capacity factor
            marginal_cost=0) 

network.add("Generator", 
            "gas", 
            bus="my bus", 
            p_nom=20, 
            marginal_cost=60) #EUR/MWh_elec
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_nom_set,p_min_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
name,,,,,,,,,,,,,,,,,,,,,
solar,my bus,PQ,,15.0,0.0,False,0.0,inf,NaN,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
wind,my bus,PQ,,20.0,0.0,False,0.0,inf,NaN,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gas,my bus,PQ,,20.0,0.0,False,0.0,inf,NaN,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


We add the load

In [5]:
network.add("Load", 
            "electricity demand", 
            bus="my bus", 
            p_set=13.2)
network.loads

,bus,carrier,type,p_set,q_set,sign,active
name,,,,,,,
electricity demand,my bus,,,13.2,0.0,-1.0,True


Calculate the optimal economic dispatch

In [6]:
network.optimize()

Index(['my bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.15s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 3 primals, 7 duals
Objective: 2.43e+02
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-whn1l2lr has 7 rows; 3 cols; 9 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [6e+01, 6e+01]
  Bound   [0e+00, 0e+00]
  RHS     [3e+00, 2e+01]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-7); columns 0(-3); nonzeros 0(-9) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-whn1l2lr
Model status        : Optimal
Objective value     :  2.4300000000e+02
P-D objective error :  2.3344319861e-16
HiGHS run time      :          0.00


('ok', 'optimal')

Now we can show the optimal dispatch and the electricity price (marginal cost at the bus)

In [7]:
network.generators_t.p

name,solar,wind,gas
snapshot,,,
now,2.55,6.6,4.05


In [8]:
network.buses_t.marginal_price

name,my bus
snapshot,
now,60.0
